In [69]:
#==贝叶斯公式 拼写检查==
#==需求：输入一个写错的单词，提示一个可能正确的==
#原理：用先验的已经存在词库中的词频来算出现这个词的概率，
#比较其它词，选一个可能性最大的给提示出来

import re, collections


In [71]:
'''
#贝叶斯公式 P(c|w) -> P(w|c)P(c)/P(w)
# c correct 要纠正的词, w word 输入的词 
#求解上面概率公式的最大值，把那个c提示纠正出来
#注意：分母可以消掉，因为比较不同的候选词时都是一样的输入词分母，
#所以只要比较分子即可。
'''

#处理词库（随便找一篇英文）
#把词拿出来，大小转小写，去掉特输字符，只留a-z
def words(text):
    return re.findall('[a-z]+',text.lower())


#词频统计，即P(c)先验概率
#（词频：词出现的次数,频率次数，不是概率，不用除整体的词数）
def train(features):
    model = collections.defaultdict(lambda:1)
    #lambda 定定一个返回1的匿名函数，给collection.defaultdict
    #特征字典功能是指定值并返回
    #此处意为指定词频默认设成1，
    #避免语料库中不存在的词，出现P(h)是0，结果无效的情况 model +=1累加
    #即，词最少出现一次，没出现也算一次
    for f in features:
        model[f] +=1
        
    return model

#统计全部checkWord.text文件（语料库）中出现的词频（词出现的次数）
NWORDS = train(words(open(r'data/checkWord.txt').read()))

#打印词频（先验概率P（c））
print(NWORDS)
    

defaultdict(<function train.<locals>.<lambda> at 0x000000F1C85BAF28>, {'a': 41, 'simple': 2, 'illustration': 3, 'of': 49, 'the': 63, 'clash': 2, 'and': 40, 'confusion': 2, 'two': 4, 'opposing': 4, 'systems': 6, 'memories': 4, 'in': 31, 'dreams': 11, 'when': 4, 'due': 2, 'apperceptive': 2, 'control': 2, 'is': 15, 'lacking': 2, 'supplied': 2, 'by': 7, 'common': 2, 'well': 2, 'recognised': 2, 'type': 4, 'dream': 18, 'returning': 3, 'to': 35, 'school': 5, 'youth': 2, 'many': 2, 'people': 2, 'are': 6, 'occasionally': 2, 'liable': 2, 'this': 9, 'which': 11, 'often': 3, 'vivid': 4, 'disturbing': 2, 'we': 5, 'may': 4, 'have': 5, 'left': 2, 'schoolroom': 2, 'thirty': 3, 'years': 3, 'or': 4, 'more': 7, 'ago': 2, 'never': 2, 'seen': 3, 'it': 12, 'since': 3, 'vanished': 2, 'from': 7, 'our': 7, 'waking': 2, 'thoughts': 2, 'yet': 5, 'time': 7, 'find': 3, 'ourselves': 2, 'there': 7, 'called': 2, 'upon': 3, 'take': 3, 'old': 8, 'place': 4, 'always': 2, 'with': 6, 'sense': 4, 'conflict': 2, 'vague': 2,

In [82]:
#下面来算P(w|c)
#根据 编辑距离（即，使用了几次变换后能从正确词变到输错的那个词，
#比如，the输成了tha，只有一个字母错了，那编辑距离就是1
#距离越大变换越复杂，说明越不可能，越小的越有可能犯这个错
#可能性概率大，为0则表示完全没错，
#所以优选编辑距离小越常见错的时候的这个候选词）

alphabet = 'abcdefghijklmnopqrstuvwxyz'

#==返回一个集合与输入词编辑距离 = 1 的所有词集合==
#（即，只变换一次就能输错的这些词，
#比如只输入错一个字母e，输入成了a，或多加了一个字母，或多删了一个字母，增删改） 

def edits1(word):
    n = len(word)
    return set([word[0:i]+word[i+1:] for i in range(n)]+
        [word[0:i]+word[i+1]+word[i]+word[i+2:] for i in range(n-1)]+
        [word[0:i]+c+word[i+1:] for i in range(n) for c in alphabet]+
         [word[0:i]+c+word[i:] for i in range(n+1) for c in alphabet]
        )
#第一行：删，遍历词长度,删掉第i位的字符，
#左取右不取，i被漏掉了，只留了i之前的和i之后的字母
#print('test'[0:1]) #t 
#print('test'[2:]) #st
#上面组合，tst删了第i位
#第二行：有一个字母换位了
#print('abcdefg'[0:1]+'abcdefg'[2]+'abcdefg'[1]+'abcdefg'[3:])
#acbdefg
#第三行，写错一个字母，漏掉i，换成c，任意字母了
#第三行，多加了一个字母
#print("==编辑距离为1的词集合==")
#print(edits1('appy'))

#编辑距离为2的词集合（在1的基础上再打乱一次，就是距离为2的）
def edits2(word):
    return set(e2 for e1 in edits1(word) for e2 in edits1(e1))
#print("==编辑距离为2的词集合==")
#print(edits2('appy'))
#{'arpiy', 'hpty', 'appyqy', 'apypg', 'yaspy', 'appywy', 'amppuy', 'cappt', 'qaxppy', 'haapy', 'zppyr', 'appuo', 'appqyq', 'oapdpy', 'apqi', 'azpmpy', 'asppo', 'wppr', 'lakppy', 'axppyu', 'bspy', 'ampzy', 'agpby', 'afkppy', 'mappn', 'nlappy', 'anppyw', 'qappu', 'akkppy', 'valpy', 'dppy', 'alppcy', 'aany', 'xakpy', 'rapqpy', 'xpiy', 'apyty', 'hagpy', 'napepy', 'jfpy', 

In [80]:
#测试打印发现编辑复杂度为1和2的有十分多的组合，但大多数都是错的，只有几个词是正确的词
#{'any', 'happy', 'away'}
#这些集合是我们用来纠正的候选词，所以只需保留正确的，错的过滤掉。

#过滤的方法就是保留在语料库中的词
#重新定义edits2
def known_edits2(word):
    return set(e2 for e1 in edits1(word) for e2 in edits1(e1) if e2 in NWORDS)
print(known_edits2('appy'))#测试发现只保留了正确的候选词，不正确的都过滤掉了
#只算编辑距离为1和2的先比较，如果为1的概率大就直接选中即可，不用再去算更不可能的编辑距离3了


{'any', 'happy', 'away'}


In [89]:
#返回在词库中的词的词集合
def known(words):
    return set(w for w in words if w in NWORDS)
#known(['at','of','something','filter'])#{'at', 'of', 'something'}

#取NWORDS字典中的v最大值的key
def correct(word):
    #比较编辑距离为0 为1 为2 
    #也即概率P(w|c)（把c输成w的可能性）为0 1 2 时 的 词库中最大词频P(c),
    #即两个条件组合P(w|c)*P(c)决定选哪个正确词推荐 
    candidates = known([word]) or known(edits1(word)) or known_edits2(word) or [word]
    #known([word])和词本身一致，距离为0;
    #known(edits1(word))距离为1的词；
    #known_edits2(word)距离为2的词
    #[word]不在词库中的词
    print("候选词：",candidates)
    #有了候选结果选一个最大的就好了
    return max(candidates, key = lambda w: NWORDS[w])

#max取最大值测试，key在词库中的最大词频
#print("==静态词测试==")
#max(['at','of','something'],key = lambda w: NWORDS[w])#'of'
#print(NWORDS['at'],NWORDS['of'],NWORDS['something'])#7 49 2
#print(max({'the', 'to', 'th', 'two', 'too'},key = lambda w: NWORDS[w]))#the
#print(NWORDS['the'],NWORDS['to'],NWORDS['th'],NWORDS['two'],NWORDS['too'])#63 35 3 4 2

print("==纠正以下输错的词（考虑双因素P(w|c)P(c)））==\n")
print("morw 纠正：",correct('morw'))#more
print("appl 纠正：",correct('appl'))#happy
print("tho 纠正：",correct('tho'))#the
print("somethin 纠正”：",correct('somethin'))#something


==纠正以下输错的词==

候选词： {'more'}
morw 纠正： more
候选词： {'happy', 'all'}
appl 纠正： happy
候选词： {'the', 'to', 'th', 'two', 'too'}
tho 纠正： the
候选词： {'something'}
somethin 纠正”： something


In [96]:
#下面看下不考虑P(w|c)正确词和输错词之间概率可能性大小，只考虑P(c)词频大小时的纠正效果
def correct_2(word):
    candidates_2 = NWORDS
    #print("候选词2：",candidates_2)
    #以整个词库做了候选词：{'a': 41, 'simple': 2, 'illustration': 3, 'of': 49, 'the': 63, 'clash': 2, 'and': 40, 'confusion': 2, 'two': 4, 'opposing': 4, 'systems': 6, 'memories': 4, 'in': 31, 'dreams': 11, 'when': 4, 'due': 2, 'apperceptive': 2, 'control': 2, 'is': 15, 'lacking': 2, 'supplied': 2, 'by': 7, 'common': 2, 'well': 2, 'recognised': 2
    return max(candidates_2, key = lambda w: NWORDS[w])

print("==反例：纠正以下输错的词（只考虑单因素P(c)））==\n")
print("morw 纠正：",correct_2('morw'))#more
print("appl 纠正：",correct_2('appl'))#happy
print("tho 纠正：",correct_2('tho'))#the
print("somethin 纠正”：",correct_2('somethin'))#something

#结果很显然没有达到纠正的目的，因为它是以整个词库做为候选词了
#所以选出来的最大词频肯定是和输入的词是不搭边的，选中了词库中最大词频the推荐了
#所在，要在词库众多的词中，找到跟这个错词有关的长的差不多的词，从这些当中选最大词频的才可以
#找相关词本例就是通过编辑距离来算的，
#距离越小相关度越高，选几个最小的和P(c)在词库中的词频概率相结合就是上文中正确的纠正结果
#只考虑P(w|c)呢，也是不行的，那个只是算了可能的词表，脱离了P(c)，它就没有一个词频属性和衡量标准，没有办法比较大小。
#所以要P(w|c)P(c)二者结合


==反例：纠正以下输错的词（只考虑单因素P(c)））==

morw 纠正： the
appl 纠正： the
tho 纠正： the
somethin 纠正”： the


In [ ]:
'''
1、贝叶斯原理：求逆向概率，通过已经的先验概率求解自然界中的未知概率
公式：P（A|B） = P(B|A)P(A)/P(B)
比如上例P（c正确词|w输入的错词），我们要求这个输错词条件下是某正词库中的正确词的概率，
但这个条件输入的错词，是有各种可能的，我们无法知道。
所以求这样的一个未知的概率，就可以通过，P（w|c）P(c)/P(w) 这个公式，
转化为求P（w|c）的过程，P(c)是已知的，正确词在词库中的词频，是先验概率。
#P（w）输入词在比较同一个词的多个推荐词可能性大小时，都是这同一个输入，所以只需比较分子，分母可以消掉。
P（w|c）是可求的，给定条件c正确的词，求输错成w的可能性大小，
方法是通过编辑距离（变换复杂度）给出一些侯选c词表，选复杂度最小最可能出错的，
再结合每个key对应的 P(c)，正确词在词库中的词频, 选一个最大可能的，来推荐纠正


2、朴素贝叶斯：
垃圾邮件原理：和拼写检查的应用原理一样，不同的就是它会为每个词做检查，看是否符合垃圾邮件的条件
P(h+|D) = P(D|h+)P(h+)/P(D)
同样的转换为求P(D|h+)
#D是输入的很多词集; h+是垃圾邮件条件
P(D|h+) = P(d1,d2,d3...|h+)#很多个输入
h+条件同时都出现在每个d里面过于严苛，只要有一个满足就应该是垃圾邮件
所以上式可展开为：
P(d1|h+)*P(d2|d1,h+)*P(d3|d2,d1,h+)#保留前面词对当前词的影响即可

朴素贝叶斯（也叫简单贝叶斯simple...）与贝叶斯的区别就是：
假设每一个特征d都是独立的，d1与d2 d2与d3之间的影响是不存在的。
所以上式不需考虑d2|d1 只考虑d2， d3|d2只考虑d3即可。
化简为：
P(d1|h+)*P(d2|h+)*P(d3|h+)
#每一个输入词是垃圾代入总式
P(h+|D) = P(D|h+)P(h+)/P(D)
即可算出整个邮件是垃圾邮件的概率

'''